In [1]:
import sys
import os
import PIL
from PIL import Image
import psutil
import time

sys.path.append(os.path.abspath("../"))
from src.helper import rgb_to_cmyk, calculate_cmyk_percentage, predict_pdf, label_to_price

In [2]:
initial_cpu = psutil.cpu_percent(interval=None)
initial_memory = psutil.virtual_memory().used

img = Image.open("../outputs/pdfium_10dpi/page-001.jpg").convert("RGB")
final_cpu = psutil.cpu_percent(interval=None)
final_memory = psutil.virtual_memory().used

# Menghitung perubahan
cpu_usage = final_cpu - initial_cpu
memory_usage = final_memory - initial_memory

# print(f"Waktu eksekusi: {end_time - start_time:.4f} detik")
print(f"Penggunaan CPU: {cpu_usage:.2f}%")
print(f"Penggunaan memori: {memory_usage / (1024 ** 2):.2f} MB") 

Penggunaan CPU: 30.50%
Penggunaan memori: 0.77 MB


In [109]:
import pandas as pd

In [123]:
df = pd.read_csv("../datasets/cmyk-by-dpi.csv")
df

,page,dpi,processing_time,c,m,y,k,sum
0,1,300,0.915638,60.78,1.99,52.04,43.14,157.95
1,2,300,0.886884,0.00,0.00,0.00,0.07,0.07
2,3,300,0.909601,3.49,0.16,9.08,2.96,15.69
3,4,300,0.902281,0.00,0.16,0.09,2.74,2.99
4,5,300,0.888387,0.00,0.11,0.05,3.87,4.03
...,...,...,...,...,...,...,...,...
7663,848,5,0.001077,0.00,0.00,0.00,0.09,0.09
7664,849,5,0.000960,1.15,0.30,1.81,4.22,7.48
7665,850,5,0.000798,1.11,0.32,1.90,3.90,7.23
7666,851,5,0.000722,0.00,0.00,0.00,0.09,0.09


In [124]:
df['sum'] = df['c'] + df['m'] + df['y'] + df['k']
df

,page,dpi,processing_time,c,m,y,k,sum
0,1,300,0.915638,60.78,1.99,52.04,43.14,157.95
1,2,300,0.886884,0.00,0.00,0.00,0.07,0.07
2,3,300,0.909601,3.49,0.16,9.08,2.96,15.69
3,4,300,0.902281,0.00,0.16,0.09,2.74,2.99
4,5,300,0.888387,0.00,0.11,0.05,3.87,4.03
...,...,...,...,...,...,...,...,...
7663,848,5,0.001077,0.00,0.00,0.00,0.09,0.09
7664,849,5,0.000960,1.15,0.30,1.81,4.22,7.48
7665,850,5,0.000798,1.11,0.32,1.90,3.90,7.23
7666,851,5,0.000722,0.00,0.00,0.00,0.09,0.09


In [125]:
df300 = df[df['dpi'] == 300].copy()
df300

,page,dpi,processing_time,c,m,y,k,sum
0,1,300,0.915638,60.78,1.99,52.04,43.14,157.95
1,2,300,0.886884,0.00,0.00,0.00,0.07,0.07
2,3,300,0.909601,3.49,0.16,9.08,2.96,15.69
3,4,300,0.902281,0.00,0.16,0.09,2.74,2.99
4,5,300,0.888387,0.00,0.11,0.05,3.87,4.03
...,...,...,...,...,...,...,...,...
847,848,300,0.886205,0.00,0.00,0.00,0.07,0.07
848,849,300,0.885225,1.03,0.11,1.82,4.19,7.15
849,850,300,0.896352,1.04,0.08,1.81,3.93,6.86
850,851,300,0.881693,0.00,0.00,0.00,0.07,0.07


In [126]:
diff_of_sum300dpi = []
for row in df.iterrows():
    # print(row[1]['page'],"||", df300[df300['page'] == row[1]['page']]['sum'].iloc[0])
    print(row)
    # print(df300[df300['page'] == row[1]['page']]['page'])
    sum_300 = df300[df300['page'] == row[1]['page']]['sum'].iloc[0]
    temp_sum = row[1]['sum']
    # print(temp_sum, sum_300)
    # break

    # print(temp_sum, sum_300)
    diff = sum_300 - temp_sum
    if diff < 0:
        diff_of_sum300dpi.append(diff * -1)
    else:
        diff_of_sum300dpi.append(diff)
    # break
# diff_of_sum300dpi

(0, page                 1.000000
dpi                300.000000
processing_time      0.915638
c                   60.780000
m                    1.990000
y                   52.040000
k                   43.140000
sum                157.950000
Name: 0, dtype: float64)
(1, page                 2.000000
dpi                300.000000
processing_time      0.886884
c                    0.000000
m                    0.000000
y                    0.000000
k                    0.070000
sum                  0.070000
Name: 1, dtype: float64)
(2, page                 3.000000
dpi                300.000000
processing_time      0.909601
c                    3.490000
m                    0.160000
y                    9.080000
k                    2.960000
sum                 15.690000
Name: 2, dtype: float64)
(3, page                 4.000000
dpi                300.000000
processing_time      0.902281
c                    0.000000
m                    0.160000
y                    0.090000
k        

In [128]:
df['diff'] = diff_of_sum300dpi
df

,page,dpi,processing_time,c,m,y,k,sum,diff
0,1,300,0.915638,60.78,1.99,52.04,43.14,157.95,0.00
1,2,300,0.886884,0.00,0.00,0.00,0.07,0.07,0.00
2,3,300,0.909601,3.49,0.16,9.08,2.96,15.69,0.00
3,4,300,0.902281,0.00,0.16,0.09,2.74,2.99,0.00
4,5,300,0.888387,0.00,0.11,0.05,3.87,4.03,0.00
...,...,...,...,...,...,...,...,...,...
7663,848,5,0.001077,0.00,0.00,0.00,0.09,0.09,0.02
7664,849,5,0.000960,1.15,0.30,1.81,4.22,7.48,0.33
7665,850,5,0.000798,1.11,0.32,1.90,3.90,7.23,0.37
7666,851,5,0.000722,0.00,0.00,0.00,0.09,0.09,0.02


In [137]:
df[(df['diff'] > 1) & (df['dpi'] >= 50)]

,page,dpi,processing_time,c,m,y,k,sum,diff
3416,9,100,0.114393,4.70,2.44,10.33,7.70,25.17,1.30
4267,8,50,0.011293,2.95,0.28,6.13,6.00,15.36,2.50
4268,9,50,0.011335,4.66,2.69,10.61,9.35,27.31,3.44
4374,115,50,0.011666,19.12,1.51,35.87,18.50,75.00,1.15
4612,353,50,0.021524,33.88,1.20,33.32,26.14,94.54,1.24
4694,435,50,0.011777,21.22,2.23,39.41,25.06,87.92,1.16
4708,449,50,0.011838,12.53,8.43,27.64,15.86,64.46,1.08
4734,475,50,0.012496,21.34,7.00,37.57,23.65,89.56,1.87
4760,501,50,0.011993,27.49,5.43,33.74,23.22,89.88,1.32
4914,655,50,0.011884,34.67,1.67,34.71,30.16,101.21,1.28
